Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

1. Откройте файл с данными. Посмотрите где и в каком количестве есть пропуски. Строки, где пропуски по 2 и более признакам удалите (т.е. это строки под номерами 3 и 271).

In [ ]:
# Load the uploaded dataset to examine its structure
data = pd.read_csv('./penguins.csv', encoding='latin1')

# Display general information about the data, including missing values
data_info = data.info()
missing_values = data.isnull().sum()

data_info, missing_values

2. Разделите весь набор данных на 2 части: часть без пропусков data и часть data_with_nan, где имеются пропуски в столбце "sex" 

In [ ]:
# Разделение данных на две части: одна без пропусков в столбце 'sex', другая с пропусками
data_clean = data[data['sex'].notna()]  # Данные без пропусков в столбце 'sex'
data_with_nan = data[data['sex'].isna()]  # Данные с пропусками в столбце 'sex'

# Проверка размеров новых датафреймов
data_clean_info = data_clean.info()
data_with_nan_info = data_with_nan.info()

data_clean_info, data_with_nan_info

3. Решите задачу классификации пингвинов по полу методом KNN на наборе данных data:

отделите целевой признак от нецелевых
разделите данные на части для обучения train и тестирования test
проведите кодирование категориальных признаков в train, а затем в test используя кодировщики, обученные на train
проведите нормирование нецелевых признаков в train, а затем в test используя нормировщик, обученный на train
проведите подбор гиперпараметров (количество соседей и метрика расстояния) в KNN
обучите классификатор с наилучшими гиперпараметрами на train
проверьте его качество на test

In [ ]:
# 1. Отделяем целевой признак от нецелевых
X = data_clean.drop(columns=['sex'])  # Все признаки, кроме 'sex'
y = data_clean['sex']  # Целевой признак - 'sex'

# 2. Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Кодируем категориальные признаки (например, 'island' и 'species') в X_train и X_test
# Создаём кодировщик
encoder = LabelEncoder()

# Применяем кодирование для каждого категориального признака в train и test
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

for column in X_train_encoded.select_dtypes(include=['object']).columns:
    X_train_encoded[column] = encoder.fit_transform(X_train_encoded[column])
    X_test_encoded[column] = encoder.transform(X_test_encoded[column])

# 4. Нормируем данные (например, числовые признаки)
scaler = StandardScaler()

# Применяем нормализацию для числовых признаков в X_train и X_test
X_train_scaled = X_train_encoded.copy()
X_test_scaled = X_test_encoded.copy()

X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

# 5. Подбор гиперпараметров для KNN с использованием GridSearchCV
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn = KNeighborsClassifier()

# Ищем наилучшие гиперпараметры
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# Лучшие гиперпараметры
best_params = grid_search.best_params_

# 6. Обучаем классификатор с наилучшими гиперпараметрами на train
best_knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], metric=best_params['metric'])
best_knn.fit(X_train_scaled, y_train)

# 7. Проверяем качество модели на тестовой выборке
y_pred = best_knn.predict(X_test_scaled)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)

accuracy

4. Используя обученный классификатор заполните пропуски пола в  data_with_nan. Предварительно надо

 у data_with_nan  удалить столбец "sex"; 
провести кодирование нецелевых признаков обученным  на train кодировщиком; 
провести нормировку обученным на train нормализатором. 
После этого предскажите обученным классификатором значение целевого признака на данных их data_with_nan

In [ ]:
# 1. Удаляем столбец 'sex' из data_with_nan
data_with_nan_cleaned = data_with_nan.drop(columns=['sex'])

# 2. Кодируем нецелевые признаки, используя обученный кодировщик
data_with_nan_encoded = data_with_nan_cleaned.copy()

for column in data_with_nan_encoded.select_dtypes(include=['object']).columns:
    data_with_nan_encoded[column] = encoder.transform(data_with_nan_encoded[column])

# 3. Нормируем данные, используя обученный нормализатор
data_with_nan_scaled = scaler.transform(data_with_nan_encoded)

# 4. Предсказываем значение целевого признака (пол) для пропусков
y_pred_nan = best_knn.predict(data_with_nan_scaled)

# 5. Заполняем пропуски в столбце 'sex' в data_with_nan предсказанными значениями
data_with_nan['sex'] = y_pred_nan

# Проверяем результаты
data_with_nan.head()

5. Объедините нецелевые столбцы из data_with_nan и предсказанные значения пола, выведите их. Сравните с заполнением, которое мы делали, ориентируясь на вес пингвина

In [ ]:
# 1. Извлекаем нецелевые столбцы из data_with_nan
non_target_columns = data_with_nan.drop(columns=['sex'])

# 2. Объединяем нецелевые столбцы с предсказанными значениями пола
data_with_predictions = non_target_columns.copy()
data_with_predictions['predicted_sex'] = y_pred_nan

# 3. Выводим первые несколько строк для проверки
data_with_predictions.head()

# 4. Сравнение с заполнением на основе веса пингвина (предполагаем, что столбец с весом называется 'body_mass_g')
# Сортируем по весу и заполняем пропуски
data_with_nan_sorted = data_with_nan.sort_values(by='body_mass_g', ascending=False)
data_with_nan_sorted['sex_filled_by_mass'] = data_with_nan_sorted['sex'].fillna(method='bfill')

# 5. Сравниваем результаты заполнения
comparison = data_with_nan_sorted[['body_mass_g', 'sex_filled_by_mass', 'sex']].head(10)

comparison
